<a href="https://colab.research.google.com/github/julesripoll/insa-n7-labs/blob/main/TP_PointNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Aujourd'hui, vous êtes dans la peau d'un des Data Scientists les plus renommés du pays. Vous travaillez pour une entreprise internationale de sécurité. Sa mission est de reconnaître les avions circulant dans l'espace aérien du centre spatial de Toulouse Matabiau.

Vous êtes chapeauté par le docteur Henri Muller et faites partie d'une équipe faisant elle-même partie d'un groupe de trois équipes chargées de mettre au point une nouvelle méthode de reconnaissance des avions, basée sur l'utilisation de nuages de points.

La première équipe a pour mission de réaliser la segmentation sémantique de scans de l'espace aérien afin d'isoler les avions du reste du nuage de points. Elle a depuis longtemps obtenu des résultats très satisfaisants, qui constituent votre base de données.

La seconde autre équipe a déjà obtenu des résultats très probants sur la reconnaissance d'avion grâce à l'étude de ses parties (taille des ailes, nombre de moteurs, etc.). 

Quand à votre équipe, son rôle est d'assurer la découpage en parties de l'avion à partir de son nuage de points. Mais elle est à la traine... Est-ce la faute aux nombreux afterworks un peu trop arrosés de ces derniers mois ? (/!\ Ce sujet de TP a été écrit avant la pandémie de Covid-19 et les mesures sanitaires qui ont suivi, et ne consiste en aucun cas à une incitation à braver les directives gouvernementales. Appliquez les gestes barrières.) Vous ne préférez pas y penser. En effet, la deadline est demain. 

Votre travail devrait ainsi permettre de faire la liaison entre les travaux de la première et de la seconde équipe et ainsi de terminer ce projet essentiel dans le développement de votre entreprise.

Après une petite recherche bibliographique, votre équipe a découvert une approche par apprentissage profond qui semble prometteuse : le réseau de neurones PointNet. En effet, selon ses auteurs, il est très performant dans le découpage en parties de formes 3D représentées par un nuage de points.
![PointNet](http://stanford.edu/~rqi/pointnet/images/teaser.jpg)

L'architecture du modèle complet est illustrée ci-dessous.
![Réseaux](http://stanford.edu/~rqi/pointnet/images/pointnet.jpg)

Vous avez donc passé quelques jours à annoter péniblement des nuages de points afin de constituer une base d'apprentissage.




# ATTENTION 
 Pour le bon déroulement du TP, il vous faut enlever le GPU sinon votre RAM ne va pas supporter la quantité de données !!
 Allez dans Execution > Modifier le type d'execution > Sélectionner None 

# Chargement de la base d'apprentissage
Commencez d'abord par ajouter le répertoire contenant le dataset à votre Drive :

https://drive.google.com/open?id=10KTcmOFkhNcGVwWBKHzzinK5Y6OjsNO3



Montez ensuite votre répertoire Google Drive afin de pouvoir y accéder :

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# ATTENTION
La ligne de code suivante est à DECOMMENTER une seule fois.
Ensuite il faut le RECOMMENTER.
Cette ligne vous permet d'éviter cette erreur lors du chargement du submodel : 

"AttributeError: 'str' object has no attribute 'decode'"

L'execution peut finir par une erreur mais ce n'est pas grave tant que le package h5py a été installé sous sa version 2.10.


In [ ]:
#!pip install h5py==2.10.0 --force-reinstall

Chargez ensuite les fichiers *airplane_train.h5* contenant les données d'entraînement et *airplane_test.h5* contenant les données de test :

In [ ]:
import h5py
import numpy as np

def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    return (data, label)

# Nombre de point par échantillon
num_points = 1024

# Nombre de catégories
k = 4

# Chargement des données d'entraînement
filename = '/content/drive/My Drive/PointNet/airplane_train.h5'
train_points, train_labels = load_h5(filename)
train_points = train_points.reshape(-1, num_points, 3)
train_labels = train_labels.reshape(-1, num_points, k)

# Chargement des données de test
filename = '/content/drive/My Drive/PointNet/airplane_test.h5'
test_points, test_labels = load_h5(filename)
test_points = test_points.reshape(-1, num_points, 3)
test_labels = test_labels.reshape(-1, num_points, k)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """


# Parce que c'est votre projet !

Il n'y a plus qu'à implémenter et entraîner le réseau ! Un de vos collègues s'est déjà chargé de la partie classification du modèle (i.e. la partie avec le fond bleu), cependant il vient d'aller se coucher, épuisé après ses deux heures de travail hebdomadaires.


In [ ]:
%tensorflow_version 1.x
from keras.layers import Input
from keras.layers import Convolution1D, BatchNormalization, MaxPooling1D
from keras.layers import Dense, Reshape
from keras.layers import Lambda, concatenate
from keras.models import Model

import tensorflow as tf

def mat_mul(A, B):
    return tf.matmul(A, B)


def exp_dim(global_feature, num_points):
    return tf.tile(global_feature, [1, num_points, 1])

# Modèle pour la classification + concatenation des features locales et globales
def get_submodel():
  # Input Transformation Net
  input_points = Input(shape=(num_points, 3))
  # T-Net
  x = Convolution1D(64, 1, activation='relu',
                  input_shape=(num_points, 3))(input_points)
  x = BatchNormalization()(x)
  x = Convolution1D(128, 1, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Convolution1D(1024, 1, activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPooling1D(pool_size=num_points)(x)
  x = Dense(512, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dense(256, activation='relu')(x)
  x = BatchNormalization()(x)

  # 3*3 Transform
  x = Dense(9, weights=[np.zeros([256, 9]), np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32)])(x)
  input_T = Reshape((3, 3))(x)

  # Multiplication matricielle
  g = Lambda(mat_mul, arguments={'B': input_T})(input_points)

  # MLP(64,64)
  g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)
  g = BatchNormalization()(g)
  g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)
  g = BatchNormalization()(g)

  # Feature Transformation Net
  # T-Net
  f = Convolution1D(64, 1, activation='relu')(g)
  f = BatchNormalization()(f)
  f = Convolution1D(128, 1, activation='relu')(f)
  f = BatchNormalization()(f)
  f = Convolution1D(1024, 1, activation='relu')(f)
  f = BatchNormalization()(f)
  f = MaxPooling1D(pool_size=num_points)(f)
  f = Dense(512, activation='relu')(f)
  f = BatchNormalization()(f)
  f = Dense(256, activation='relu')(f)
  f = BatchNormalization()(f)

  # 64*64 Transform
  f = Dense(64 * 64, weights=[np.zeros([256, 64 * 64]), np.eye(64).flatten().astype(np.float32)])(f)
  feature_T = Reshape((64, 64))(f)

  # Multiplication matricielle
  g = Lambda(mat_mul, arguments={'B': feature_T})(g)
  seg_part1 = g

  # MLP(64,128,1024)
  g = Convolution1D(64, 1, activation='relu')(g)
  g = BatchNormalization()(g)
  g = Convolution1D(128, 1, activation='relu')(g)
  g = BatchNormalization()(g)
  g = Convolution1D(1024, 1, activation='relu')(g)
  g = BatchNormalization()(g)

  # Global Feature
  global_feature = MaxPooling1D(pool_size=num_points)(g)
  global_feature = Lambda(exp_dim, arguments={'num_points': num_points})(global_feature)

  conc = concatenate([seg_part1, global_feature])

  model = Model(inputs = input_points, outputs = conc, name="submodel")
  model.load_weights('/content/drive/My Drive/PointNet/submodel.h5')

  return model

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
tf.version.VERSION

'1.15.2'

Pendant ce temps, vous avez astucieusement utilisé votre temps et votre moteur de recherche favori, ce qui vous a permis de trouver le code de deux fonctions utilisées pour réaliser de l'augmentation de données.

In [ ]:
# Fonction auxiliaire de rotation des points
def rotate_point_cloud(batch_data):
    rotated_data = np.zeros(batch_data.shape, dtype=np.float32)
    for k in range(batch_data.shape[0]):
        rotation_angle = np.random.uniform() * 2 * np.pi
        cosval = np.cos(rotation_angle)
        sinval = np.sin(rotation_angle)
        rotation_matrix = np.array([[cosval, 0, sinval],
                                    [0, 1, 0],
                                    [-sinval, 0, cosval]])
        shape_pc = batch_data[k, ...]
        rotated_data[k, ...] = np.dot(shape_pc.reshape((-1, 3)), rotation_matrix)
    return rotated_data

# Fonction auxiliaire de bruitage des points
def jitter_point_cloud(batch_data, sigma=0.01, clip=0.05):
    B, N, C = batch_data.shape
    assert(clip > 0)
    jittered_data = np.clip(sigma * np.random.randn(B, N, C), -1 * clip, clip)
    jittered_data += batch_data
    return jittered_data




La seule tâche restant à réaliser est le réseau de segmentation (i.e. la partie avec le fond jaune), et c'est votre projet ! Toute votre équipe et toute votre entreprise compte sur vous !

Le réseau que vous devez implémenter doit donc prendre en entrée les prédictions du réseau précédent et déterminer la classe à laquelle appartient chaque point du nuage de points. 

Il n'est pas demandé d'entraîner le réseau fourni par votre collègue, l'apprentissage ayant déjà été effectué auparavant, mais seulement d'entraîner le réseau que vous allez implémenter

In [ ]:
# Ne pas oublier d'importer les bons élements de la bibliothèse Keras

# Construction du modèle pour la segmentation
"""on implémente la partie jaune. get submodel permet d'obtenir la concaténation à l'entrée du modele
 mais on la fout pas ici mais après là on se préoccupe juste des dimensions."""
def segmenter(input_shape=(num_points,1088)):
    input_points = Input(shape=input_shape)
    # MLP(512,256)
    # à implementer 


    # MLP(128,m)
    # à implementer 
    
    model = ...
    return model

In [ ]:
# Nombre d'epochs
epo = 8

# Chargement du modèle de classification 
submodel = ...

# Construction des données de test
test_points_segmenter = ...

In [ ]:


# Compilation du modèle
model = segmenter()
model.summary()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Sauvegarder du meilleur model <=> Celui qui a la loss la plus petite
# Initialisation
best_model = ...
best_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

best_score_loss = 100

for i in range(epo):
    # Rotation et bruitage des points
    # à implementer 

    # Construction des données
    latent = ...

    # Entraînement
    # à implementer 

    # Pour des questions de RAM
    del latent

    # Évaluation du modèle & sauvegarde du meilleur model
    score = model.evaluate(test_points_segmenter, test_labels, verbose=1)
    print('Test loss: ', score[0])
    if score[0] < best_score_loss:
        # à implementer 
        
    print('Test accuracy: ', score[1])

IndentationError: ignored

Comparaison entre le model après entrainement et le meilleur model

In [ ]:
import keras
score = model.evaluate(test_points_segmenter, test_labels, verbose=1)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

best_model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

score = best_model.evaluate(test_points_segmenter, test_labels, verbose=1)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Visualisation
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
v_points = test_points[1:2,:,:]
pred = model.predict(submodel.predict(v_points))
pred = np.squeeze(pred)
v_points = np.squeeze(v_points)
pred = pred.tolist()
color = ['b', 'g', 'r', 'k']
m= ['o', 'v', '<', '>']
for i in range(v_points.shape[0]):
    xs = v_points[i,0]
    ys = v_points[i,1]
    zs = v_points[i,2]
    ind = pred[i].index(max(pred[i]))
    ax.scatter(xs, ys, zs, c=color[ind], marker=m[ind])
  
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.xlim(-1,1)
plt.ylim(-1,1)
plt.show()